In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,  # Rotate images slightly
    width_shift_range=0.2,  # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,  # Shear effect
    zoom_range=0.2,  # Zoom into images
    horizontal_flip=True  # Flip images horizontally
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    'Dataset',  # Path to dataset
    target_size=(224, 224),  # Ensure input matches model shape
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Dataset',  # Path to dataset
    target_size=(224, 224),  # Ensure input matches model shape
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step 4: Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile the model for multi-class classification
model.compile(
    loss='sparse_categorical_crossentropy',  # Change from binary to categorical loss
    optimizer='adam',
    metrics=['accuracy']
)


train_steps_per_epoch = int(np.ceil(train_generator.samples / train_generator.batch_size))
validation_steps_per_epoch = int(np.ceil(validation_generator.samples / validation_generator.batch_size))

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch,
    verbose=2
)


In [ ]:
tf.keras.backend.clear_session(
    free_memory=True
)


In [ ]:
# Save the model
model.save('check.h5')


In [ ]:
print(model.input_shape)


In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

# Load the trained model
model_path = 'check.h5'  # Ensure the model file exists
if not os.path.exists(model_path):
    print(f"❌ Error: Model file '{model_path}' not found.")
    exit(1)

try:
    model = load_model(model_path)
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit(1)

# Function to preprocess and predict currency authenticity
def predict_currency(image_path):
    """ Reads an image file, preprocesses it, and predicts if it's REAL or FAKE. """
    try:
        if not os.path.exists(image_path):
            return f"❌ Error: File '{image_path}' does not exist."

        # Load and preprocess the image
        img = Image.open(image_path).convert('RGB')  # Ensure 3 color channels
        img = img.resize((224, 224))  # Resize to match model input size
        img = img_to_array(img) / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img)[0][0]  # Extract prediction value
        confidence = prediction * 100  # Convert to percentage
        label = "REAL" if prediction > 0.5 else "FAKE"

        return f"🔹 Prediction: {label} ({confidence:.2f}% confidence)"
    
    except Exception as e:
        return f"❌ Error processing image: {e}"


# Ask for an image file path
image_path = input("Enter the image file path: ").strip()

if image_path:
    result = predict_currency(image_path)
    print(result)
else:
    print("❌ No file path provided.")
